# Sampling algorithms in `sbi`

Note: this tutorial requires that the user is already familiar with the [flexible interface](https://sbi-dev.github.io/sbi/tutorial/02_flexible_interface/).

`sbi` implements three methods: SNPE, SNLE, and SNRE. When using SNPE, the trained neural network directly approximates the posterior. Thus, sampling from the posterior can be done by sampling from the trained neural network. The neural networks trained in SNLE and SNRE approximate the likelihood(-ratio). Thus, in order to draw samples from the posterior, one has to perform additional sampling steps, e.g. Markov-chain Monte-Carlo (MCMC). In `sbi`, the implemented samplers are:

- Markov-chain Monte-Carlo (MCMC)

- Rejection sampling

- Variational inference (VI)

Below, we will demonstrate how these samplers can be used in `sbi`. First, we train the neural network:


In [ ]:
# imports
import torch
from sbi.inference import SNLE

## Defining simulator, prior, and running inference

Our simulator (model) accepts 2 parameters ($\theta$) and outputs simulations of the same dimensionality, adding Gaussian noise to the parameter set. For each dimension of $\theta$, we use a Gaussian _prior_ with a mean of 0 and a standard deviation of 1.

In [ ]:
num_dim = 2
prior = torch.distributions.MultivariateNormal(torch.zeros(num_dim), torch.eye(num_dim))


In [ ]:
# create inference object. Here, SNLE is used.
inference = SNLE(prior=prior, show_progress_bars=False)

# generate simulations, pass to the inference object and train the likelihood estimator
theta = prior.sample((1000,))
x = theta + torch.randn((1000, num_dim))
likelihood_estimator = inference.append_simulations(theta, x).train()

# generate the first observation
x_o = torch.randn((1, num_dim))

To specify the desired sampling technique, we provide its configuration to the `build_posterior()` method:

In [2]:
# sampling with MCMC
sampling_algorithm = "mcmc"
mcmc_method = "slice_np"  # alternatives: nuts,hmc
posterior = inference.build_posterior(sample_with=sampling_algorithm,
                                      mcmc_method=mcmc_method)

# sampling with variational inference
sampling_algorithm = "vi"
vi_method = "rKL"  # alternative: fKL
posterior = inference.build_posterior(sample_with=sampling_algorithm,
                                      vi_method=vi_method)

# unlike other methods, vi needs a training step for every observation.
posterior = posterior.set_default_x(x_o).train()

# sampling with rejection sampling
sampling_algorithm = "rejection"
posterior = inference.build_posterior(sample_with=sampling_algorithm)

  0%|          | 0/2000 [00:00<?, ?it/s]


Converged with loss: 5.17
Quality Score: -0.116 	 Good: Smaller than 0.5  Bad: Larger than 1.0 	         NOTE: Less sensitive to mode collapse.


# More flexibility in adjusting the sampler


The above syntax enables straightforward experimentation with various sampling algorithms. For tailored sampling, you can modify sampler hyperparameters, such as the number of warm-up steps for MCMC, or develop a custom sampler from scratch.

## Main syntax (for SNLE and SNRE)

As above, we begin by training the neural network.


Then, for full flexibility in using the sampler, we do not use the `.build_posterior()` method. Instead, we explicitly define the potential function and the sampling algorithm:


In [3]:
from sbi.inference import MCMCPosterior, likelihood_estimator_based_potential

# define potential function and initizialize MCMC posterior object
potential_fn, parameter_transform = likelihood_estimator_based_potential(
    likelihood_estimator, prior, x_o
)
posterior = MCMCPosterior(
    potential_fn, proposal=prior, theta_transform=parameter_transform, warmup_steps=10
)

The _posterior_ is proportional to the product of likelihood and _prior_:                                               𝑝(𝜃|𝑥𝑜)∝𝑝(𝑥𝑜|𝜃)𝑝(𝜃). A _potential function_ is a function of the parameter 𝑓(𝜃) and defined as the logarithm of the right-hand side of this equation: 𝑓(𝜃)=log(𝑝(𝑥𝑜|𝜃)𝑝(𝜃)).

If you want to use variational inference or rejection sampling, just replace the last line with `VIPosterior` or `RejectionPosterior`:


In [4]:
from sbi.inference import RejectionPosterior, VIPosterior

# define potential function and initizialize rejection posterior object
posterior = VIPosterior(
    potential_fn, prior=prior, theta_transform=parameter_transform
).train()

posterior = RejectionPosterior(
    potential_fn, proposal=prior, theta_transform=parameter_transform
)

  0%|          | 0/2000 [00:00<?, ?it/s]


Converged with loss: 5.18
Quality Score: 0.104 	 Good: Smaller than 0.5  Bad: Larger than 1.0 	         NOTE: Less sensitive to mode collapse.


At this point, you could also plug the `potential_fn` into any sampler of your choice and not rely on any of the in-built `sbi`-samplers.


## Further explanation

The first lines are the same as for the flexible interface:


In [5]:
# create inference object using SNLE, generate simulations and pass to the inference object 
inference = SNLE()
likelihood_estimator = inference.append_simulations(theta, x).train()

 Neural network successfully converged after 59 epochs.

In [2]:
# define potential function 
potential_fn, parameter_transform = likelihood_estimator_based_potential(
    likelihood_estimator, prior, x_o
)

NameError: name 'likelihood_estimator_based_potential' is not defined

By calling the `potential_fn`, you can evaluate the potential:


In [7]:
# assuming that your parameters are 1D:
potential = potential_fn(
    torch.zeros(1, num_dim)
)  # -> returns f(0) = log( p(x_o|0) p(0) )

The other object that is returned by `likelihood_estimator_based_potential` is a `parameter_transform`. The `parameter_transform` is a [pytorch transform](https://github.com/pytorch/pytorch/blob/master/torch/distributions/transforms.py). The `parameter_transform` is a fixed transform that is can be applied to parameter `theta`. It transforms the parameters into unconstrained space (if the prior is bounded, e.g. `BoxUniform`), and standardizes the parameters (i.e. zero mean, one std). Using `parameter_transform` during sampling is optional, but it usually improves the performance of MCMC.


In [8]:
# transform parameters into unconstrained space
theta_tf = parameter_transform(torch.zeros(1, num_dim))
theta_original = parameter_transform.inv(theta_tf)
print(theta_original)  # -> tensor([[0.0]])

tensor([[0., 0.]])


After having obtained the `potential_fn`, we can sample from the posterior with MCMC or rejection sampling:


In [9]:
posterior = MCMCPosterior(
    potential_fn, proposal=prior, theta_transform=parameter_transform
)
posterior = RejectionPosterior(potential_fn, proposal=prior)

## Main syntax for SNPE

SNPE usually does not require MCMC or rejection sampling (if you still need it, you can use the same syntax as above with the `posterior_estimator_based_potential` function). Instead, SNPE samples from the neural network. If the support of the prior is bounded, some samples can lie outside of the support of the prior. The `DirectPosterior` class automatically rejects these samples:


In [10]:
from sbi.inference import SNPE, DirectPosterior

inference = SNPE()
posterior_estimator = inference.append_simulations(theta, x).train()

posterior = DirectPosterior(posterior_estimator, prior=prior)

 Neural network successfully converged after 76 epochs.